In [1]:
import json
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.stats import linregress
from pprint import pprint
from config import api_key

In [2]:
injury_df = pd.read_csv("injury_data.csv")
injury_df

,Unnamed: 0,Player ID,Season,Season Type,Week,Name,Team,Position,Body Part,Declared Inactive
0,0,6048,2012,1,1,Early Doucet,ARI,WR,--,False
1,1,6353,2012,1,1,Adam Snyder,ARI,G,Elbow,False
2,2,9319,2012,1,1,LaRod Stephens-Howling,ARI,RB,Groin,False
3,3,9323,2012,1,1,Chris Wells,ARI,RB,Hamstring,False
4,4,9782,2012,1,1,Gregory Toler,ARI,CB,Hip,False
...,...,...,...,...,...,...,...,...,...,...
55613,55613,21755,2020,1,7,Isaiah Wright,WAS,WR,Shoulder,True
55614,55614,21875,2020,1,7,James Smith-Williams,WAS,DE,Concussion,True
55615,55615,21955,2020,1,7,Thaddeus Moss,WAS,TE,Undisclosed,True
55616,55616,22008,2020,1,7,Saahdiq Charles,WAS,OL,Knee,True


In [3]:
injury_df['Body Part'].value_counts()

Knee                     10828
Ankle                     5801
Hamstring                 4226
Shoulder                  3851
Foot                      2619
                         ...  
Rib, Sternum                 1
Sports hernia                1
Right shoulder, ankle        1
Knee, back                   1
Ear                          1
Name: Body Part, Length: 503, dtype: int64

In [4]:
concussion_df = injury_df.loc[(injury_df['Body Part'] == 'Concussion') & (injury_df['Body Part'] == 'Concussion, ankle')\
                              & (injury_df['Body Part'] == 'Concussion, eye') & (injury_df['Body Part'] == 'Concussion, hip')\
                              & (injury_df['Body Part'] == 'Concussion, illness') & (injury_df['Body Part'] == 'Concussion, stinger')\
                              & (injury_df['Body Part'] == 'Concussion, elbow') & (injury_df['Body Part'] == 'Concussion, knee, ankle')\
                              & (injury_df['Body Part'] == 'Concussion, hamstring') & (injury_df['Body Part'] == 'Concussion, finger')\
                              & (injury_df['Body Part'] == 'Concussion, chest') & (injury_df['Body Part'] == 'Concussion, left shoulder')\
                              & (injury_df['Body Part'] == 'Concussion, toe') & (injury_df['Body Part'] == 'Concussion, foot, thigh')\
                              & (injury_df['Body Part'] == 'Concussion, shoulder') & (injury_df['Body Part'] == 'Concussion, groin')\
                              & (injury_df['Body Part'] == 'Concussion, not injury related') & (injury_df['Body Part'] == 'Concussion, neck')\
                              & (injury_df['Body Part'] == 'Concussion, shoulder') & (injury_df['Body Part'] == 'Head')]
concussion_df

,Unnamed: 0,Player ID,Season,Season Type,Week,Name,Team,Position,Body Part,Declared Inactive


In [5]:
concussion_df = injury_df.loc[(injury_df['Body Part'] == 'Concussion')]
concussion_df

,Unnamed: 0,Player ID,Season,Season Type,Week,Name,Team,Position,Body Part,Declared Inactive
75,75,8853,2012,1,1,Gerald Sensabaugh,DAL,DB,Concussion,False
89,89,14201,2012,1,1,Bill Bentley,DET,DB,Concussion,False
100,100,14608,2012,1,1,Terrell Manning,GB,LB,Concussion,False
164,164,12762,2012,1,1,Daniel Thomas,MIA,RB,Concussion,False
224,224,6304,2012,1,1,Darrelle Revis,NYJ,DB,Concussion,False
...,...,...,...,...,...,...,...,...,...,...
55416,55416,20971,2020,1,7,Andrew Van Ginkel,MIA,OLB,Concussion,False
55438,55438,16851,2020,1,7,Carl Davis Jr.,NE,DT,Concussion,True
55456,55456,17457,2020,1,7,Nick Easton,NO,OL,Concussion,True
55473,55473,19200,2020,1,7,C.J. Board,NYG,WR,Concussion,True


In [6]:
injury_df['Body Part'] = injury_df['Body Part'].replace(
    {'Concussion, ankle': 'Concussion', 'Concussion, eye': 'Concussion', 'Concussion, hip': 'Concussion', 'Concussion, illness': 'Concussion',\
    'Concussion, stinger': 'Concussion', 'Concussion, elbow': 'Concussion', 'Concussion, knee, ankle': 'Concussion','Concussion, hamstring': 'Concussion',\
    'Concussion, finger': 'Concussion', 'Concussion, chest': 'Concussion', 'Concussion, left shoulder': 'Concussion',\
    'Concussion, toe': 'Concussion', 'Concussion, foot, thigh': 'Concussion', 'Concussion, shoulder': 'Concussion',\
    'Concussion, groin': 'Concussion','Concussion, not injury related': 'Concussion', 'Concussion, neck': 'Concussion',\
    'Concusison': 'Concussion', 'Head': 'Concussion'})

In [7]:
concussion_df = injury_df.loc[(injury_df['Body Part'] == 'Concussion')]
concussion_df

,Unnamed: 0,Player ID,Season,Season Type,Week,Name,Team,Position,Body Part,Declared Inactive
15,15,8638,2012,1,1,Sean Considine,BAL,S,Concussion,False
33,33,12607,2012,1,1,Lorenzo Booker,CHI,RB,Concussion,False
44,44,9956,2012,1,1,Dan Skuta,CIN,LB,Concussion,False
75,75,8853,2012,1,1,Gerald Sensabaugh,DAL,DB,Concussion,False
89,89,14201,2012,1,1,Bill Bentley,DET,DB,Concussion,False
...,...,...,...,...,...,...,...,...,...,...
55416,55416,20971,2020,1,7,Andrew Van Ginkel,MIA,OLB,Concussion,False
55438,55438,16851,2020,1,7,Carl Davis Jr.,NE,DT,Concussion,True
55456,55456,17457,2020,1,7,Nick Easton,NO,OL,Concussion,True
55473,55473,19200,2020,1,7,C.J. Board,NYG,WR,Concussion,True


In [8]:
injury_df

,Unnamed: 0,Player ID,Season,Season Type,Week,Name,Team,Position,Body Part,Declared Inactive
0,0,6048,2012,1,1,Early Doucet,ARI,WR,--,False
1,1,6353,2012,1,1,Adam Snyder,ARI,G,Elbow,False
2,2,9319,2012,1,1,LaRod Stephens-Howling,ARI,RB,Groin,False
3,3,9323,2012,1,1,Chris Wells,ARI,RB,Hamstring,False
4,4,9782,2012,1,1,Gregory Toler,ARI,CB,Hip,False
...,...,...,...,...,...,...,...,...,...,...
55613,55613,21755,2020,1,7,Isaiah Wright,WAS,WR,Shoulder,True
55614,55614,21875,2020,1,7,James Smith-Williams,WAS,DE,Concussion,True
55615,55615,21955,2020,1,7,Thaddeus Moss,WAS,TE,Undisclosed,True
55616,55616,22008,2020,1,7,Saahdiq Charles,WAS,OL,Knee,True


In [12]:
bins = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
bin_names = ["2011", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2021"]
injury_df['year_bins'] = pd.cut(injury_df['Season'], bins, labels = bin_names)
injury_df['year_bins']
# group_year = injury_df.groupby('year_bins')

# injuries_per_year = injury_df['Body Part'].groupby('year_bins')
# conc_per_year = injury_df[injury_df['Body Part'] == 'Concussion'].groupby('year_bins')

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
55613    2019
55614    2019
55615    2019
55616    2019
55617    2019
Name: year_bins, Length: 55618, dtype: category
Categories (9, object): [2011 < 2013 < 2014 < 2015 ... 2017 < 2018 < 2019 < 2021]

In [ ]:
df = pd.DataFrame({"Injuries Per Year": injuries_per_year,
                  "Concussions per Year": conc_per_year})
df